In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import time
import os

In [ ]:
import os
import shutil
import scipy.io
import torch
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split, DataLoader

if os.path.exists('flowers_sorted'):
    shutil.rmtree('flowers_sorted')

labels_path = 'flowers/flowers-102/imagelabels.mat'
source_dir = 'flowers/flowers-102/jpg'
target_dir = 'flowers_sorted'

labels = scipy.io.loadmat(labels_path)['labels'][0]

os.makedirs(target_dir, exist_ok=True)
for i in range(len(labels)):
    label = labels[i]
    class_dir = os.path.join(target_dir, str(label))
    os.makedirs(class_dir, exist_ok=True)
    image_filename = f'image_{i+1:05d}.jpg'
    source_path = os.path.join(source_dir, image_filename)
    target_path = os.path.join(class_dir, image_filename)
    if os.path.exists(source_path):
        shutil.copy(source_path, target_path)

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

full_dataset = ImageFolder(root='flowers_sorted', transform=data_transforms['train'])
full_dataset_val = ImageFolder(root='flowers_sorted', transform=data_transforms['val'])

total_size = len(full_dataset)
train_size = int(0.8 * total_size)
val_size = int(0.1 * total_size)
test_size = total_size - train_size - val_size

print(f"\nTotal images: {total_size}")
print(f"Splitting into: Train: {train_size}, Validation: {val_size}, Test: {test_size}")

generator = torch.Generator().manual_seed(42)
train_indices, val_indices, test_indices = random_split(range(total_size), [train_size, val_size, test_size], generator=generator)

train_dataset = torch.utils.data.Subset(full_dataset, train_indices)
val_dataset = torch.utils.data.Subset(full_dataset_val, val_indices)
test_dataset = torch.utils.data.Subset(full_dataset_val, test_indices)

batch_size = 32
dataloaders = {
    'train': DataLoader(train_dataset, batch_size=batch_size, shuffle=True),
    'val': DataLoader(val_dataset, batch_size=batch_size, shuffle=False),
    'test': DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
}
dataset_sizes = { 'train': len(train_dataset), 'val': len(val_dataset), 'test': len(test_dataset) }

Removing old 'flowers_sorted' directory...
Organizing files...
File organization complete!

Total images: 8189
Splitting into -> Train: 6551, Validation: 818, Test: 820

DataLoaders created successfully with the new 80/10/10 split!


In [ ]:
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

class CNN(nn.Module):
    def __init__(self, num_classes=102):
        super(DeeperCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2, 2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1), nn.ReLU(), nn.MaxPool2d(2, 2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 14 * 14, 1024), nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(1024, num_classes)
        )
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CNN(num_classes=102).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = StepLR(optimizer, step_size=7, gamma=0.1)


Model, Loss, and Optimizer are ready.


In [ ]:
import time
import copy
import numpy as np

def train_model(model, criterion, optimizer, scheduler, dataloaders, dataset_sizes, num_epochs=25, patience=5):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_val_loss = np.inf
    patience_counter = 0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}' + '\n' + '-'*10)
        for phase in ['train', 'val']:
            model.train() if phase == 'train' else model.eval()
            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)
                optimizer.zero_grad()
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'val':
                if epoch_loss < best_val_loss:
                    best_val_loss = epoch_loss
                    best_model_wts = copy.deepcopy(model.state_dict())
                    patience_counter = 0
                else:
                    patience_counter += 1

        scheduler.step()

        if patience_counter >= patience:
            print(f"Early stopping triggered after {patience} epochs.")
            break
        print()

    time_elapsed = time.time() - since
    print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Loss: {best_val_loss:4f}')
    model.load_state_dict(best_model_wts)
    return model

trained_model = train_model(model, criterion, optimizer, scheduler, dataloaders, dataset_sizes, num_epochs=25, patience=5)

Epoch 1/25
----------
train Loss: 4.0952 Acc: 0.0696
val Loss: 3.5977 Acc: 0.1125

Epoch 2/25
----------
train Loss: 3.5242 Acc: 0.1330
val Loss: 3.1693 Acc: 0.1993

Epoch 3/25
----------
train Loss: 3.2269 Acc: 0.1902
val Loss: 2.8499 Acc: 0.2738

Epoch 4/25
----------
train Loss: 2.9928 Acc: 0.2354
val Loss: 2.5810 Acc: 0.3411

Epoch 5/25
----------
train Loss: 2.8040 Acc: 0.2754
val Loss: 2.3808 Acc: 0.3704

Epoch 6/25
----------
train Loss: 2.6273 Acc: 0.3132
val Loss: 2.1552 Acc: 0.4267

Epoch 7/25
----------
train Loss: 2.4710 Acc: 0.3457
val Loss: 2.0767 Acc: 0.4352

Epoch 8/25
----------
train Loss: 2.3747 Acc: 0.3770
val Loss: 2.0125 Acc: 0.4939

Epoch 9/25
----------
train Loss: 2.2572 Acc: 0.4007
val Loss: 1.8482 Acc: 0.5024

Epoch 10/25
----------
train Loss: 2.1556 Acc: 0.4328
val Loss: 1.7681 Acc: 0.5330

Epoch 11/25
----------
train Loss: 2.1130 Acc: 0.4296
val Loss: 1.6508 Acc: 0.5672

Epoch 12/25
----------
train Loss: 2.0018 Acc: 0.4631
val Loss: 1.6377 Acc: 0.5660

E